### 以下代码用来实现经典的编码器－解码器代码，并使用 cmn_eng　2.2w条中英文翻译数据，作为实例
如果希望更详细的解读，强烈推荐阅读：https://zhuanlan.zhihu.com/p/28054589

In [1]:
import tensorflow as tf
import os

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

In [2]:
print(tf.__version__)

2.0.0


In [3]:
import unicodedata
import re
import numpy as np
import os
import io
import time

In [4]:
with open('./data/cmn-eng.txt', 'r', encoding='utf-8') as f:
    contexts = f.readlines()
contexts[10]

'Got it?\t你懂了吗？\n'

In [6]:
def data_pro(contexts):
    processed_contexts_en = []
    processed_contexts_ch = []
    for line in contexts:
        en, ch = re.split(r'\t', line.strip())
#         print(en, ch)
        en = re.sub(r'([\?\.\!\,¿])', r' \1', en)
#         print(en)
        en = re.sub(r'\s+', ' ', en)
        ch = re.sub(r'\s+', ' ', ch)
        en = re.sub(r'[^a-zA-Z\?\.\!\,。？！，、¿]+', r' ', en)
        ch = re.sub(r'[^a-zA-Z\?\.\!\,。？！，、¿\u4e00-\u9fa5]+', r' ', ch)
        en = '<start> ' + en + ' <end>'
        ch = '<start> ' + ' '.join([i for i in ch]) + ' <end>'
        
        processed_contexts_en.append(en)
        processed_contexts_ch.append(ch)
    
    return processed_contexts_en, processed_contexts_ch

max_len_en = set()
max_len_ch = set()

def get_vocab(processed_contexts_en, processed_contexts_ch):
    en_vocab = set()
    ch_vocab = set()
    for en, ch in zip(processed_contexts_en, processed_contexts_ch):
        len_en = len([w for w in re.split(r'\s', en)])
        len_ch = len([w for w in re.split(r'\s', ch)])
        max_len_en.add(len_en)
        max_len_ch.add(len_ch)
        
        en_vocab.update(set([w for w in re.split(r'\s', en)]))
        ch_vocab.update(set([w for w in re.split(r'\s', ch)]))

    return en_vocab, ch_vocab


processed_contexts_en, processed_contexts_ch = data_pro(contexts)
en_vocab, ch_vocab = get_vocab(processed_contexts_en, processed_contexts_ch)
print(len(en_vocab))
print(len(ch_vocab))

print('max_len_ch: ', max(max_len_ch))
print('max_len_en: ', max(max_len_en))

6927
3424
max_len_ch:  51
max_len_en:  38


In [7]:
en_vocab2id = {i:k for k,i in enumerate(en_vocab)}
ch_vocab2id = {i:k for k,i in enumerate(ch_vocab)}
id2en_vocab = {k:i for k,i in enumerate(en_vocab)}
id2ch_vocab = {k:i for k,i in enumerate(ch_vocab)}


en_vocab2id.get('hello')

592

In [8]:
# train_data, test_data = train_test_split(pro_data, test_size=0.15,shuffle=True)
train_x, test_x, train_y, test_y = train_test_split(processed_contexts_en, processed_contexts_ch, test_size=0.15)
print('train_data 数据大小：', len(train_x))
print('test_data 数据大小：', len(train_y))

train_y[-1], train_x[-1]

train_data 数据大小： 18763
test_data 数据大小： 18763


('<start> 你 不 會 遲 到 ， 是 嗎 ？ <end>',
 '<start> You won t be late , will you ? <end>')

In [11]:
def data_to_tensor(x,y):
    x = 1
    y = 1
    return tf.constant(1),tf.constant(1)

train_db = tf.data.Dataset.from_tensor_slices(
    (train_x, train_y)).map(data_to_tensor).batch(64)

#### 本编码器采用 Bahdanau 注意力

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self):
        pass
    
    def call(self):
        pass
    
    def initialize_hidden_state(self):
        return

In [ ]:
class BahdanauAttention(tf.keras.layers):
    def __init__(self):
        pass
    
    def call(self):
        pass


In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self):
        pass
    
    def call(self):
        passs
        
